In [ ]:
#Deep Feature 1: VGG16

import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, roc_auc_score, recall_score, precision_score, f1_score
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

# Load VGG16 model pre-trained on ImageNet without top layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
model = Model(inputs=base_model.input, outputs=x)

# Prepare data generators
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(
    'Enter your path to train folder',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    shuffle=False)

test_generator = test_datagen.flow_from_directory(
    'Enter your path to test folder',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    shuffle=False)

# Feature extraction
def extract_features(generator, model):
    features = model.predict(generator, verbose=1)
    labels = generator.classes
    return features, labels

train_features, train_labels = extract_features(train_generator, model)
test_features, test_labels = extract_features(test_generator, model)

# Encode labels
le = LabelEncoder()
train_labels_enc = le.fit_transform(train_labels)
test_labels_enc = le.transform(test_labels)

# Define classifiers
classifiers = {
    'Naive Bayes': GaussianNB(),
    'Linear SVM': SVC(kernel='linear', probability=True),
    'Quadratic SVM': SVC(kernel='poly', degree=2, probability=True),
    'Cubic SVM': SVC(kernel='poly', degree=3, probability=True),
    'Fine Gaussian SVM': SVC(kernel='rbf', gamma='scale', C=0.1, probability=True),
    'Medium Gaussian SVM': SVC(kernel='rbf', gamma='scale', C=1, probability=True),
    'Coarse Gaussian SVM': SVC(kernel='rbf', gamma='scale', C=10, probability=True),
    'Fine KNN': KNeighborsClassifier(n_neighbors=1),
    'Medium KNN': KNeighborsClassifier(n_neighbors=5),
    'Coarse KNN': KNeighborsClassifier(n_neighbors=10),
    'Cosine KNN': KNeighborsClassifier(n_neighbors=5, metric='cosine'),
    'Cubic KNN': KNeighborsClassifier(n_neighbors=5, metric='minkowski', p=3),
    'Weighted KNN': KNeighborsClassifier(n_neighbors=5, weights='distance')
}

# Initialize results storage
results = {name: {'accuracy': [], 'auc': [], 'recall': [], 'precision': [], 'f1': []} for name in classifiers.keys()}
all_run_results = []

# Perform classification and record metrics
for name, clf in classifiers.items():
    print(f"Training and evaluating {name}...")
    for run in tqdm(range(10)):
        clf.fit(train_features, train_labels_enc)
        pred_labels = clf.predict(test_features)
        pred_probs = clf.predict_proba(test_features)[:, 1] if hasattr(clf, "predict_proba") else pred_labels

        accuracy = accuracy_score(test_labels_enc, pred_labels)
        auc = roc_auc_score(test_labels_enc, pred_probs)
        recall = recall_score(test_labels_enc, pred_labels)
        precision = precision_score(test_labels_enc, pred_labels)
        f1 = f1_score(test_labels_enc, pred_labels)

        results[name]['accuracy'].append(accuracy)
        results[name]['auc'].append(auc)
        results[name]['recall'].append(recall)
        results[name]['precision'].append(precision)
        results[name]['f1'].append(f1)

        all_run_results.append({
            'classifier': name,
            'run': run + 1,
            'accuracy': accuracy,
            'auc': auc,
            'recall': recall,
            'precision': precision,
            'f1': f1
        })

# Calculate mean and standard deviation
summary = {}
for name, metrics in results.items():
    summary[name] = {metric: {'mean': np.mean(values), 'std': np.std(values)} for metric, values in metrics.items()}

# Convert summary to DataFrame
summary_df = pd.DataFrame({(clf, metric): vals for clf, metrics in summary.items() for metric, vals in metrics.items()})
print(summary_df)

# Convert all run results to DataFrame
all_run_results_df = pd.DataFrame(all_run_results)

# Save results to CSV
summary_df.to_csv('classification_summary_VGG16.csv')
all_run_results_df.to_csv('all_run_results_VGG16.csv')

In [ ]:
#Deep Feature 2: VGG19

import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, roc_auc_score, recall_score, precision_score, f1_score
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg19 import VGG19, preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

# Load VGG19 model pre-trained on ImageNet without top layers
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
model = Model(inputs=base_model.input, outputs=x)

# Prepare data generators
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(
    'Enter your path to train folder',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    shuffle=False)

test_generator = test_datagen.flow_from_directory(
    'Enter your path to test folder',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    shuffle=False)

# Feature extraction
def extract_features(generator, model):
    features = model.predict(generator, verbose=1)
    labels = generator.classes
    return features, labels

train_features, train_labels = extract_features(train_generator, model)
test_features, test_labels = extract_features(test_generator, model)

# Encode labels
le = LabelEncoder()
train_labels_enc = le.fit_transform(train_labels)
test_labels_enc = le.transform(test_labels)

# Define classifiers
classifiers = {
    'Naive Bayes': GaussianNB(),
    'Linear SVM': SVC(kernel='linear', probability=True),
    'Quadratic SVM': SVC(kernel='poly', degree=2, probability=True),
    'Cubic SVM': SVC(kernel='poly', degree=3, probability=True),
    'Fine Gaussian SVM': SVC(kernel='rbf', gamma='scale', C=0.1, probability=True),
    'Medium Gaussian SVM': SVC(kernel='rbf', gamma='scale', C=1, probability=True),
    'Coarse Gaussian SVM': SVC(kernel='rbf', gamma='scale', C=10, probability=True),
    'Fine KNN': KNeighborsClassifier(n_neighbors=1),
    'Medium KNN': KNeighborsClassifier(n_neighbors=5),
    'Coarse KNN': KNeighborsClassifier(n_neighbors=10),
    'Cosine KNN': KNeighborsClassifier(n_neighbors=5, metric='cosine'),
    'Cubic KNN': KNeighborsClassifier(n_neighbors=5, metric='minkowski', p=3),
    'Weighted KNN': KNeighborsClassifier(n_neighbors=5, weights='distance')
}

# Initialize results storage
results = {name: {'accuracy': [], 'auc': [], 'recall': [], 'precision': [], 'f1': []} for name in classifiers.keys()}
all_run_results = []

# Perform classification and record metrics
for name, clf in classifiers.items():
    print(f"Training and evaluating {name}...")
    for run in tqdm(range(10)):
        clf.fit(train_features, train_labels_enc)
        pred_labels = clf.predict(test_features)
        pred_probs = clf.predict_proba(test_features)[:, 1] if hasattr(clf, "predict_proba") else pred_labels

        accuracy = accuracy_score(test_labels_enc, pred_labels)
        auc = roc_auc_score(test_labels_enc, pred_probs)
        recall = recall_score(test_labels_enc, pred_labels)
        precision = precision_score(test_labels_enc, pred_labels)
        f1 = f1_score(test_labels_enc, pred_labels)

        results[name]['accuracy'].append(accuracy)
        results[name]['auc'].append(auc)
        results[name]['recall'].append(recall)
        results[name]['precision'].append(precision)
        results[name]['f1'].append(f1)

        all_run_results.append({
            'classifier': name,
            'run': run + 1,
            'accuracy': accuracy,
            'auc': auc,
            'recall': recall,
            'precision': precision,
            'f1': f1
        })

# Calculate mean and standard deviation
summary = {}
for name, metrics in results.items():
    summary[name] = {metric: {'mean': np.mean(values), 'std': np.std(values)} for metric, values in metrics.items()}

# Convert summary to DataFrame
summary_df = pd.DataFrame({(clf, metric): vals for clf, metrics in summary.items() for metric, vals in metrics.items()})
print(summary_df)

# Convert all run results to DataFrame
all_run_results_df = pd.DataFrame(all_run_results)

# Save results to CSV
summary_df.to_csv('classification_summary_vgg19.csv')
all_run_results_df.to_csv('all_run_results_vgg19.csv')

In [ ]:
#Deep Feature 3: Resnet50

import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, roc_auc_score, recall_score, precision_score, f1_score
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

# Load ResNet50 model pre-trained on ImageNet without top layers
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
model = Model(inputs=base_model.input, outputs=x)

# Prepare data generators
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(
    'Enter your path to train folder',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    shuffle=False)

test_generator = test_datagen.flow_from_directory(
    'Enter your path to test folder',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    shuffle=False)

# Feature extraction
def extract_features(generator, model):
    features = model.predict(generator, verbose=1)
    labels = generator.classes
    return features, labels

train_features, train_labels = extract_features(train_generator, model)
test_features, test_labels = extract_features(test_generator, model)

# Encode labels
le = LabelEncoder()
train_labels_enc = le.fit_transform(train_labels)
test_labels_enc = le.transform(test_labels)

# Define classifiers
classifiers = {
    'Naive Bayes': GaussianNB(),
    'Linear SVM': SVC(kernel='linear', probability=True),
    'Quadratic SVM': SVC(kernel='poly', degree=2, probability=True),
    'Cubic SVM': SVC(kernel='poly', degree=3, probability=True),
    'Fine Gaussian SVM': SVC(kernel='rbf', gamma='scale', C=0.1, probability=True),
    'Medium Gaussian SVM': SVC(kernel='rbf', gamma='scale', C=1, probability=True),
    'Coarse Gaussian SVM': SVC(kernel='rbf', gamma='scale', C=10, probability=True),
    'Fine KNN': KNeighborsClassifier(n_neighbors=1),
    'Medium KNN': KNeighborsClassifier(n_neighbors=5),
    'Coarse KNN': KNeighborsClassifier(n_neighbors=10),
    'Cosine KNN': KNeighborsClassifier(n_neighbors=5, metric='cosine'),
    'Cubic KNN': KNeighborsClassifier(n_neighbors=5, metric='minkowski', p=3),
    'Weighted KNN': KNeighborsClassifier(n_neighbors=5, weights='distance')
}

# Initialize results storage
results = {name: {'accuracy': [], 'auc': [], 'recall': [], 'precision': [], 'f1': []} for name in classifiers.keys()}
all_run_results = []

# Perform classification and record metrics
for name, clf in classifiers.items():
    print(f"Training and evaluating {name}...")
    for run in tqdm(range(10)):
        clf.fit(train_features, train_labels_enc)
        pred_labels = clf.predict(test_features)
        pred_probs = clf.predict_proba(test_features)[:, 1] if hasattr(clf, "predict_proba") else pred_labels

        accuracy = accuracy_score(test_labels_enc, pred_labels)
        auc = roc_auc_score(test_labels_enc, pred_probs)
        recall = recall_score(test_labels_enc, pred_labels)
        precision = precision_score(test_labels_enc, pred_labels)
        f1 = f1_score(test_labels_enc, pred_labels)

        results[name]['accuracy'].append(accuracy)
        results[name]['auc'].append(auc)
        results[name]['recall'].append(recall)
        results[name]['precision'].append(precision)
        results[name]['f1'].append(f1)

        all_run_results.append({
            'classifier': name,
            'run': run + 1,
            'accuracy': accuracy,
            'auc': auc,
            'recall': recall,
            'precision': precision,
            'f1': f1
        })

# Calculate mean and standard deviation
summary = {}
for name, metrics in results.items():
    summary[name] = {metric: {'mean': np.mean(values), 'std': np.std(values)} for metric, values in metrics.items()}

# Convert summary to DataFrame
summary_df = pd.DataFrame({(clf, metric): vals for clf, metrics in summary.items() for metric, vals in metrics.items()})
print(summary_df)

# Convert all run results to DataFrame
all_run_results_df = pd.DataFrame(all_run_results)

# Save results to CSV
summary_df.to_csv('classification_summary_resnet50.csv')
all_run_results_df.to_csv('all_run_results_resnet50.csv')


In [ ]:
#Deep Feature 4: GoogleNet (Inception v3)

import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, roc_auc_score, recall_score, precision_score, f1_score
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

# Load InceptionV3 model pre-trained on ImageNet without top layers
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
model = Model(inputs=base_model.input, outputs=x)

# Prepare data generators
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(
    'Enter your path to train folder',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    shuffle=False)

test_generator = test_datagen.flow_from_directory(
    'Enter your path to test folder',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    shuffle=False)

# Feature extraction
def extract_features(generator, model):
    features = model.predict(generator, verbose=1)
    labels = generator.classes
    return features, labels

train_features, train_labels = extract_features(train_generator, model)
test_features, test_labels = extract_features(test_generator, model)

# Encode labels
le = LabelEncoder()
train_labels_enc = le.fit_transform(train_labels)
test_labels_enc = le.transform(test_labels)

# Define classifiers
classifiers = {
    'Naive Bayes': GaussianNB(),
    'Linear SVM': SVC(kernel='linear', probability=True),
    'Quadratic SVM': SVC(kernel='poly', degree=2, probability=True),
    'Cubic SVM': SVC(kernel='poly', degree=3, probability=True),
    'Fine Gaussian SVM': SVC(kernel='rbf', gamma='scale', C=0.1, probability=True),
    'Medium Gaussian SVM': SVC(kernel='rbf', gamma='scale', C=1, probability=True),
    'Coarse Gaussian SVM': SVC(kernel='rbf', gamma='scale', C=10, probability=True),
    'Fine KNN': KNeighborsClassifier(n_neighbors=1),
    'Medium KNN': KNeighborsClassifier(n_neighbors=5),
    'Coarse KNN': KNeighborsClassifier(n_neighbors=10),
    'Cosine KNN': KNeighborsClassifier(n_neighbors=5, metric='cosine'),
    'Cubic KNN': KNeighborsClassifier(n_neighbors=5, metric='minkowski', p=3),
    'Weighted KNN': KNeighborsClassifier(n_neighbors=5, weights='distance')
}

# Initialize results storage
results = {name: {'accuracy': [], 'auc': [], 'recall': [], 'precision': [], 'f1': []} for name in classifiers.keys()}
all_run_results = []

# Perform classification and record metrics
for name, clf in classifiers.items():
    print(f"Training and evaluating {name}...")
    for run in tqdm(range(10)):
        clf.fit(train_features, train_labels_enc)
        pred_labels = clf.predict(test_features)
        pred_probs = clf.predict_proba(test_features)[:, 1] if hasattr(clf, "predict_proba") else pred_labels

        accuracy = accuracy_score(test_labels_enc, pred_labels)
        auc = roc_auc_score(test_labels_enc, pred_probs)
        recall = recall_score(test_labels_enc, pred_labels)
        precision = precision_score(test_labels_enc, pred_labels)
        f1 = f1_score(test_labels_enc, pred_labels)

        results[name]['accuracy'].append(accuracy)
        results[name]['auc'].append(auc)
        results[name]['recall'].append(recall)
        results[name]['precision'].append(precision)
        results[name]['f1'].append(f1)

        all_run_results.append({
            'classifier': name,
            'run': run + 1,
            'accuracy': accuracy,
            'auc': auc,
            'recall': recall,
            'precision': precision,
            'f1': f1
        })

# Calculate mean and standard deviation
summary = {}
for name, metrics in results.items():
    summary[name] = {metric: {'mean': np.mean(values), 'std': np.std(values)} for metric, values in metrics.items()}

# Convert summary to DataFrame
summary_df = pd.DataFrame({(clf, metric): vals for clf, metrics in summary.items() for metric, vals in metrics.items()})
print(summary_df)

# Convert all run results to DataFrame
all_run_results_df = pd.DataFrame(all_run_results)

# Save results to CSV
summary_df.to_csv('classification_summary_inceptionv3.csv')
all_run_results_df.to_csv('all_run_results_inceptionv3.csv')

In [ ]:
#HANDCRAFTED FEATURE: HOG

import os
import numpy as np
import pandas as pd
from skimage.io import imread
from skimage.transform import resize
from skimage.feature import hog
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, roc_auc_score, recall_score, precision_score, f1_score
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

# Define paths
train_path = 'Enter your path to train folder'
test_path = 'Enter your path to test folder'

# Prepare data and labels
def prepare_data(path, img_size=(128, 128)):
    data = []
    labels = []
    for folder in os.listdir(path):
        folder_path = os.path.join(path, folder)
        if os.path.isdir(folder_path):
            for file in os.listdir(folder_path):
                file_path = os.path.join(folder_path, file)
                if file_path.endswith('.jpg') or file_path.endswith('.png'):
                    img = imread(file_path, as_gray=True)
                    img_resized = resize(img, img_size)
                    data.append(img_resized)
                    labels.append(folder)
    return np.array(data), np.array(labels)

# Extract HOG features
def extract_hog_features(data):
    features = []
    for image in data:
        hog_features = hog(image, pixels_per_cell=(8, 8), cells_per_block=(2, 2), feature_vector=True)
        features.append(hog_features)
    return np.array(features)

# Prepare training and testing data
train_data, train_labels = prepare_data(train_path)
test_data, test_labels = prepare_data(test_path)

# Extract HOG features from the training and testing data
train_features = extract_hog_features(train_data)
test_features = extract_hog_features(test_data)

# Encode labels
le = LabelEncoder()
train_labels_enc = le.fit_transform(train_labels)
test_labels_enc = le.transform(test_labels)

# Define classifiers
classifiers = {
    'Naive Bayes': GaussianNB(),
    'Linear SVM': SVC(kernel='linear', probability=True),
    'Quadratic SVM': SVC(kernel='poly', degree=2, probability=True),
    'Cubic SVM': SVC(kernel='poly', degree=3, probability=True),
    'Fine Gaussian SVM': SVC(kernel='rbf', gamma='scale', C=0.1, probability=True),
    'Medium Gaussian SVM': SVC(kernel='rbf', gamma='scale', C=1, probability=True),
    'Coarse Gaussian SVM': SVC(kernel='rbf', gamma='scale', C=10, probability=True),
    'Fine KNN': KNeighborsClassifier(n_neighbors=1),
    'Medium KNN': KNeighborsClassifier(n_neighbors=5),
    'Coarse KNN': KNeighborsClassifier(n_neighbors=10),
    'Cosine KNN': KNeighborsClassifier(n_neighbors=5, metric='cosine'),
    'Cubic KNN': KNeighborsClassifier(n_neighbors=5, metric='minkowski', p=3),
    'Weighted KNN': KNeighborsClassifier(n_neighbors=5, weights='distance')
}

# Initialize results storage
results = {name: {'accuracy': [], 'auc': [], 'recall': [], 'precision': [], 'f1': []} for name in classifiers.keys()}
all_run_results = []

# Perform classification and record metrics
for name, clf in classifiers.items():
    print(f"Training and evaluating {name}...")
    for run in tqdm(range(10)):
        clf.fit(train_features, train_labels_enc)
        pred_labels = clf.predict(test_features)
        pred_probs = clf.predict_proba(test_features)[:, 1] if hasattr(clf, "predict_proba") else pred_labels

        accuracy = accuracy_score(test_labels_enc, pred_labels)
        auc = roc_auc_score(test_labels_enc, pred_probs)
        recall = recall_score(test_labels_enc, pred_labels)
        precision = precision_score(test_labels_enc, pred_labels)
        f1 = f1_score(test_labels_enc, pred_labels)

        results[name]['accuracy'].append(accuracy)
        results[name]['auc'].append(auc)
        results[name]['recall'].append(recall)
        results[name]['precision'].append(precision)
        results[name]['f1'].append(f1)

        all_run_results.append({
            'classifier': name,
            'run': run + 1,
            'accuracy': accuracy,
            'auc': auc,
            'recall': recall,
            'precision': precision,
            'f1': f1
        })

# Calculate mean and standard deviation
summary = {}
for name, metrics in results.items():
    summary[name] = {metric: {'mean': np.mean(values), 'std': np.std(values)} for metric, values in metrics.items()}

# Convert summary to DataFrame
summary_df = pd.DataFrame({(clf, metric): vals for clf, metrics in summary.items() for metric, vals in metrics.items()})
print(summary_df)

# Convert all run results to DataFrame
all_run_results_df = pd.DataFrame(all_run_results)

# Save results to CSV
summary_df.to_csv('classification_summary_hog.csv')
all_run_results_df.to_csv('all_run_results_hog.csv')

In [ ]:
#HANDCRAFTED FEATURE: LBP

import os
import numpy as np
import pandas as pd
from skimage.io import imread
from skimage.transform import resize
from skimage.feature import local_binary_pattern
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, roc_auc_score, recall_score, precision_score, f1_score
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

# Define paths
train_path = 'Enter your path to train folder'
test_path = 'Enter your path to test folder'

# Prepare data and labels
def prepare_data(path, img_size=(128, 128)):
    data = []
    labels = []
    for folder in os.listdir(path):
        folder_path = os.path.join(path, folder)
        if os.path.isdir(folder_path):
            for file in os.listdir(folder_path):
                file_path = os.path.join(folder_path, file)
                if file_path.endswith('.jpg') or file_path.endswith('.png'):
                    img = imread(file_path, as_gray=True)
                    img_resized = resize(img, img_size)
                    data.append(img_resized)
                    labels.append(folder)
    return np.array(data), np.array(labels)

# Extract LBP features
def extract_lbp_features(data, P=8, R=1):
    features = []
    for image in data:
        lbp = local_binary_pattern(image, P=P, R=R, method='uniform')
        (hist, _) = np.histogram(lbp.ravel(), bins=np.arange(0, P + 3), range=(0, P + 2))
        hist = hist.astype("float")
        hist /= (hist.sum() + 1e-6)
        features.append(hist)
    return np.array(features)

# Prepare training and testing data
train_data, train_labels = prepare_data(train_path)
test_data, test_labels = prepare_data(test_path)

# Extract LBP features from the training and testing data
train_features = extract_lbp_features(train_data)
test_features = extract_lbp_features(test_data)

# Encode labels
le = LabelEncoder()
train_labels_enc = le.fit_transform(train_labels)
test_labels_enc = le.transform(test_labels)

# Define classifiers
classifiers = {
    'Naive Bayes': GaussianNB(),
    'Linear SVM': SVC(kernel='linear', probability=True),
    'Quadratic SVM': SVC(kernel='poly', degree=2, probability=True),
    'Cubic SVM': SVC(kernel='poly', degree=3, probability=True),
    'Fine Gaussian SVM': SVC(kernel='rbf', gamma='scale', C=0.1, probability=True),
    'Medium Gaussian SVM': SVC(kernel='rbf', gamma='scale', C=1, probability=True),
    'Coarse Gaussian SVM': SVC(kernel='rbf', gamma='scale', C=10, probability=True),
    'Fine KNN': KNeighborsClassifier(n_neighbors=1),
    'Medium KNN': KNeighborsClassifier(n_neighbors=5),
    'Coarse KNN': KNeighborsClassifier(n_neighbors=10),
    'Cosine KNN': KNeighborsClassifier(n_neighbors=5, metric='cosine'),
    'Cubic KNN': KNeighborsClassifier(n_neighbors=5, metric='minkowski', p=3),
    'Weighted KNN': KNeighborsClassifier(n_neighbors=5, weights='distance')
}

# Initialize results storage
results = {name: {'accuracy': [], 'auc': [], 'recall': [], 'precision': [], 'f1': []} for name in classifiers.keys()}
all_run_results = []

# Perform classification and record metrics
for name, clf in classifiers.items():
    print(f"Training and evaluating {name}...")
    for run in tqdm(range(10)):
        clf.fit(train_features, train_labels_enc)
        pred_labels = clf.predict(test_features)
        pred_probs = clf.predict_proba(test_features)[:, 1] if hasattr(clf, "predict_proba") else pred_labels

        accuracy = accuracy_score(test_labels_enc, pred_labels)
        auc = roc_auc_score(test_labels_enc, pred_probs)
        recall = recall_score(test_labels_enc, pred_labels)
        precision = precision_score(test_labels_enc, pred_labels)
        f1 = f1_score(test_labels_enc, pred_labels)

        results[name]['accuracy'].append(accuracy)
        results[name]['auc'].append(auc)
        results[name]['recall'].append(recall)
        results[name]['precision'].append(precision)
        results[name]['f1'].append(f1)

        all_run_results.append({
            'classifier': name,
            'run': run + 1,
            'accuracy': accuracy,
            'auc': auc,
            'recall': recall,
            'precision': precision,
            'f1': f1
        })

# Calculate mean and standard deviation
summary = {}
for name, metrics in results.items():
    summary[name] = {metric: {'mean': np.mean(values), 'std': np.std(values)} for metric, values in metrics.items()}

# Convert summary to DataFrame
summary_df = pd.DataFrame({(clf, metric): vals for clf, metrics in summary.items() for metric, vals in metrics.items()})
print(summary_df)

# Convert all run results to DataFrame
all_run_results_df = pd.DataFrame(all_run_results)

# Save results to CSV
summary_df.to_csv('classification_summary_lbp.csv')
all_run_results_df.to_csv('all_run_results_lbp.csv')

In [ ]:
#HANDCRAFTED FEATURE: GLCM

import os
import numpy as np
import pandas as pd
from skimage.io import imread
from skimage.transform import resize
from skimage.feature import graycomatrix, graycoprops
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, roc_auc_score, recall_score, precision_score, f1_score
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

# Define paths
train_path = 'Enter your path to train folder'
test_path = 'Enter your path to test folder'

# Prepare data and labels
def prepare_data(path, img_size=(128, 128)):
    data = []
    labels = []
    for folder in os.listdir(path):
        folder_path = os.path.join(path, folder)
        if os.path.isdir(folder_path):
            for file in os.listdir(folder_path):
                file_path = os.path.join(folder_path, file)
                if file_path.endswith('.jpg') or file_path.endswith('.png'):
                    img = imread(file_path, as_gray=True)
                    img_resized = resize(img, img_size)
                    data.append(img_resized)
                    labels.append(folder)
    return np.array(data), np.array(labels)

# Extract GLCM features
def extract_glcm_features(data, distances=[1], angles=[0], properties=['contrast', 'dissimilarity', 'homogeneity', 'ASM', 'energy', 'correlation']):
    features = []
    for image in data:
        glcm = graycomatrix((image * 255).astype('uint8'), distances=distances, angles=angles, symmetric=True, normed=True)
        feature_vector = []
        for prop in properties:
            feature = graycoprops(glcm, prop)
            feature_vector.append(feature.flatten())
        features.append(np.hstack(feature_vector))
    return np.array(features)

# Prepare training and testing data
train_data, train_labels = prepare_data(train_path)
test_data, test_labels = prepare_data(test_path)

# Extract GLCM features from the training and testing data
train_features = extract_glcm_features(train_data)
test_features = extract_glcm_features(test_data)

# Encode labels
le = LabelEncoder()
train_labels_enc = le.fit_transform(train_labels)
test_labels_enc = le.transform(test_labels)

# Define classifiers
classifiers = {
    'Naive Bayes': GaussianNB(),
    'Linear SVM': SVC(kernel='linear', probability=True),
    'Quadratic SVM': SVC(kernel='poly', degree=2, probability=True),
    'Cubic SVM': SVC(kernel='poly', degree=3, probability=True),
    'Fine Gaussian SVM': SVC(kernel='rbf', gamma='scale', C=0.1, probability=True),
    'Medium Gaussian SVM': SVC(kernel='rbf', gamma='scale', C=1, probability=True),
    'Coarse Gaussian SVM': SVC(kernel='rbf', gamma='scale', C=10, probability=True),
    'Fine KNN': KNeighborsClassifier(n_neighbors=1),
    'Medium KNN': KNeighborsClassifier(n_neighbors=5),
    'Coarse KNN': KNeighborsClassifier(n_neighbors=10),
    'Cosine KNN': KNeighborsClassifier(n_neighbors=5, metric='cosine'),
    'Cubic KNN': KNeighborsClassifier(n_neighbors=5, metric='minkowski', p=3),
    'Weighted KNN': KNeighborsClassifier(n_neighbors=5, weights='distance')
}

# Initialize results storage
results = {name: {'accuracy': [], 'auc': [], 'recall': [], 'precision': [], 'f1': []} for name in classifiers.keys()}
all_run_results = []

# Perform classification and record metrics
for name, clf in classifiers.items():
    print(f"Training and evaluating {name}...")
    for run in tqdm(range(10)):
        clf.fit(train_features, train_labels_enc)
        pred_labels = clf.predict(test_features)
        pred_probs = clf.predict_proba(test_features)[:, 1] if hasattr(clf, "predict_proba") else pred_labels

        accuracy = accuracy_score(test_labels_enc, pred_labels)
        auc = roc_auc_score(test_labels_enc, pred_probs)
        recall = recall_score(test_labels_enc, pred_labels)
        precision = precision_score(test_labels_enc, pred_labels)
        f1 = f1_score(test_labels_enc, pred_labels)

        results[name]['accuracy'].append(accuracy)
        results[name]['auc'].append(auc)
        results[name]['recall'].append(recall)
        results[name]['precision'].append(precision)
        results[name]['f1'].append(f1)

        all_run_results.append({
            'classifier': name,
            'run': run + 1,
            'accuracy': accuracy,
            'auc': auc,
            'recall': recall,
            'precision': precision,
            'f1': f1
        })

# Calculate mean and standard deviation
summary = {}
for name, metrics in results.items():
    summary[name] = {metric: {'mean': np.mean(values), 'std': np.std(values)} for metric, values in metrics.items()}

# Convert summary to DataFrame
summary_df = pd.DataFrame({(clf, metric): vals for clf, metrics in summary.items() for metric, vals in metrics.items()})
print(summary_df)

# Convert all run results to DataFrame
all_run_results_df = pd.DataFrame(all_run_results)

# Save results to CSV
summary_df.to_csv('classification_summary_glcm.csv')
all_run_results_df.to_csv('all_run_results_glcm.csv')

In [ ]:
#FUSION MODEL 1: VGG16 & VGG19

import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, roc_auc_score, recall_score, precision_score, f1_score
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input as preprocess_input_vgg16
from tensorflow.keras.applications.vgg19 import VGG19, preprocess_input as preprocess_input_vgg19
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

# Load VGG16 model pre-trained on ImageNet without top layers
base_model_vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x_vgg16 = base_model_vgg16.output
x_vgg16 = GlobalAveragePooling2D()(x_vgg16)
model_vgg16 = Model(inputs=base_model_vgg16.input, outputs=x_vgg16)

# Load VGG19 model pre-trained on ImageNet without top layers
base_model_vgg19 = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x_vgg19 = base_model_vgg19.output
x_vgg19 = GlobalAveragePooling2D()(x_vgg19)
model_vgg19 = Model(inputs=base_model_vgg19.input, outputs=x_vgg19)

# Prepare data generators
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input_vgg16)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input_vgg16)

train_generator = train_datagen.flow_from_directory(
    'Enter your path to train folder',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    shuffle=False)

test_generator = test_datagen.flow_from_directory(
    'Enter your path to test folder',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    shuffle=False)

# Feature extraction
def extract_features(generator, model_vgg16, model_vgg19):
    features_vgg16 = model_vgg16.predict(generator, verbose=1)
    features_vgg19 = model_vgg19.predict(generator, verbose=1)
    features = np.concatenate([features_vgg16, features_vgg19], axis=1)
    labels = generator.classes
    return features, labels

train_features, train_labels = extract_features(train_generator, model_vgg16, model_vgg19)
test_features, test_labels = extract_features(test_generator, model_vgg16, model_vgg19)

# Encode labels
le = LabelEncoder()
train_labels_enc = le.fit_transform(train_labels)
test_labels_enc = le.transform(test_labels)

# Define classifiers
classifiers = {
    'Naive Bayes': GaussianNB(),
    'Linear SVM': SVC(kernel='linear', probability=True),
    'Quadratic SVM': SVC(kernel='poly', degree=2, probability=True),
    'Cubic SVM': SVC(kernel='poly', degree=3, probability=True),
    'Fine Gaussian SVM': SVC(kernel='rbf', gamma='scale', C=0.1, probability=True),
    'Medium Gaussian SVM': SVC(kernel='rbf', gamma='scale', C=1, probability=True),
    'Coarse Gaussian SVM': SVC(kernel='rbf', gamma='scale', C=10, probability=True),
    'Fine KNN': KNeighborsClassifier(n_neighbors=1),
    'Medium KNN': KNeighborsClassifier(n_neighbors=5),
    'Coarse KNN': KNeighborsClassifier(n_neighbors=10),
    'Cosine KNN': KNeighborsClassifier(n_neighbors=5, metric='cosine'),
    'Cubic KNN': KNeighborsClassifier(n_neighbors=5, metric='minkowski', p=3),
    'Weighted KNN': KNeighborsClassifier(n_neighbors=5, weights='distance')
}

# Initialize results storage
results = {name: {'accuracy': [], 'auc': [], 'recall': [], 'precision': [], 'f1': []} for name in classifiers.keys()}
all_run_results = []

# Perform classification and record metrics
for name, clf in classifiers.items():
    print(f"Training and evaluating {name}...")
    for run in tqdm(range(10)):
        clf.fit(train_features, train_labels_enc)
        pred_labels = clf.predict(test_features)
        pred_probs = clf.predict_proba(test_features)[:, 1] if hasattr(clf, "predict_proba") else pred_labels

        accuracy = accuracy_score(test_labels_enc, pred_labels)
        auc = roc_auc_score(test_labels_enc, pred_probs)
        recall = recall_score(test_labels_enc, pred_labels)
        precision = precision_score(test_labels_enc, pred_labels)
        f1 = f1_score(test_labels_enc, pred_labels)

        results[name]['accuracy'].append(accuracy)
        results[name]['auc'].append(auc)
        results[name]['recall'].append(recall)
        results[name]['precision'].append(precision)
        results[name]['f1'].append(f1)

        all_run_results.append({
            'classifier': name,
            'run': run + 1,
            'accuracy': accuracy,
            'auc': auc,
            'recall': recall,
            'precision': precision,
            'f1': f1
        })

# Calculate mean and standard deviation
summary = {}
for name, metrics in results.items():
    summary[name] = {metric: {'mean': np.mean(values), 'std': np.std(values)} for metric, values in metrics.items()}

# Convert summary to DataFrame
summary_df = pd.DataFrame({(clf, metric): vals for clf, metrics in summary.items() for metric, vals in metrics.items()})
print(summary_df)

# Convert all run results to DataFrame
all_run_results_df = pd.DataFrame(all_run_results)

# Save results to CSV
summary_df.to_csv('classification_summary_vgg16_vgg19_fusion.csv')
all_run_results_df.to_csv('all_run_results_vgg16_vgg19_fusion.csv')

In [ ]:
#FUSION MODEL 2: VGG16 & Resnet50

import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, roc_auc_score, recall_score, precision_score, f1_score
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input as preprocess_input_vgg16
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input as preprocess_input_resnet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

# Load VGG16 model pre-trained on ImageNet without top layers
base_model_vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x_vgg16 = base_model_vgg16.output
x_vgg16 = GlobalAveragePooling2D()(x_vgg16)
model_vgg16 = Model(inputs=base_model_vgg16.input, outputs=x_vgg16)

# Load ResNet50 model pre-trained on ImageNet without top layers
base_model_resnet50 = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x_resnet50 = base_model_resnet50.output
x_resnet50 = GlobalAveragePooling2D()(x_resnet50)
model_resnet50 = Model(inputs=base_model_resnet50.input, outputs=x_resnet50)

# Prepare data generators
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input_vgg16)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input_vgg16)

train_generator = train_datagen.flow_from_directory(
    'Enter your path to train folder',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    shuffle=False)

test_generator = test_datagen.flow_from_directory(
    'Enter your path to test folder',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    shuffle=False)

# Feature extraction
def extract_features(generator, model_vgg16, model_resnet50):
    features_vgg16 = model_vgg16.predict(generator, verbose=1)
    features_resnet50 = model_resnet50.predict(generator, verbose=1)
    features = np.concatenate([features_vgg16, features_resnet50], axis=1)
    labels = generator.classes
    return features, labels

train_features, train_labels = extract_features(train_generator, model_vgg16, model_resnet50)
test_features, test_labels = extract_features(test_generator, model_vgg16, model_resnet50)

# Encode labels
le = LabelEncoder()
train_labels_enc = le.fit_transform(train_labels)
test_labels_enc = le.transform(test_labels)

# Define classifiers
classifiers = {
    'Naive Bayes': GaussianNB(),
    'Linear SVM': SVC(kernel='linear', probability=True),
    'Quadratic SVM': SVC(kernel='poly', degree=2, probability=True),
    'Cubic SVM': SVC(kernel='poly', degree=3, probability=True),
    'Fine Gaussian SVM': SVC(kernel='rbf', gamma='scale', C=0.1, probability=True),
    'Medium Gaussian SVM': SVC(kernel='rbf', gamma='scale', C=1, probability=True),
    'Coarse Gaussian SVM': SVC(kernel='rbf', gamma='scale', C=10, probability=True),
    'Fine KNN': KNeighborsClassifier(n_neighbors=1),
    'Medium KNN': KNeighborsClassifier(n_neighbors=5),
    'Coarse KNN': KNeighborsClassifier(n_neighbors=10),
    'Cosine KNN': KNeighborsClassifier(n_neighbors=5, metric='cosine'),
    'Cubic KNN': KNeighborsClassifier(n_neighbors=5, metric='minkowski', p=3),
    'Weighted KNN': KNeighborsClassifier(n_neighbors=5, weights='distance')
}

# Initialize results storage
results = {name: {'accuracy': [], 'auc': [], 'recall': [], 'precision': [], 'f1': []} for name in classifiers.keys()}
all_run_results = []

# Perform classification and record metrics
for name, clf in classifiers.items():
    print(f"Training and evaluating {name}...")
    for run in tqdm(range(10)):
        clf.fit(train_features, train_labels_enc)
        pred_labels = clf.predict(test_features)
        pred_probs = clf.predict_proba(test_features)[:, 1] if hasattr(clf, "predict_proba") else pred_labels

        accuracy = accuracy_score(test_labels_enc, pred_labels)
        auc = roc_auc_score(test_labels_enc, pred_probs)
        recall = recall_score(test_labels_enc, pred_labels)
        precision = precision_score(test_labels_enc, pred_labels)
        f1 = f1_score(test_labels_enc, pred_labels)

        results[name]['accuracy'].append(accuracy)
        results[name]['auc'].append(auc)
        results[name]['recall'].append(recall)
        results[name]['precision'].append(precision)
        results[name]['f1'].append(f1)

        all_run_results.append({
            'classifier': name,
            'run': run + 1,
            'accuracy': accuracy,
            'auc': auc,
            'recall': recall,
            'precision': precision,
            'f1': f1
        })

# Calculate mean and standard deviation
summary = {}
for name, metrics in results.items():
    summary[name] = {metric: {'mean': np.mean(values), 'std': np.std(values)} for metric, values in metrics.items()}

# Convert summary to DataFrame
summary_df = pd.DataFrame({(clf, metric): vals for clf, metrics in summary.items() for metric, vals in metrics.items()})
print(summary_df)

# Convert all run results to DataFrame
all_run_results_df = pd.DataFrame(all_run_results)

# Save results to CSV
summary_df.to_csv('classification_summary_vgg16_resnet50_fusion.csv')
all_run_results_df.to_csv('all_run_results_vgg16_resnet50_fusion.csv')

In [ ]:
#FUSION MODEL 3: VGG16 & GoogleNet(Incetion v3)

import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, roc_auc_score, recall_score, precision_score, f1_score
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input as preprocess_input_vgg16
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input as preprocess_input_inception
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

# Load VGG16 model pre-trained on ImageNet without top layers
base_model_vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x_vgg16 = base_model_vgg16.output
x_vgg16 = GlobalAveragePooling2D()(x_vgg16)
model_vgg16 = Model(inputs=base_model_vgg16.input, outputs=x_vgg16)

# Load InceptionV3 (GoogLeNet) model pre-trained on ImageNet without top layers
base_model_inception = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x_inception = base_model_inception.output
x_inception = GlobalAveragePooling2D()(x_inception)
model_inception = Model(inputs=base_model_inception.input, outputs=x_inception)

# Prepare data generators
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input_vgg16)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input_vgg16)

train_generator = train_datagen.flow_from_directory(
    'Enter your path to train folder',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    shuffle=False)

test_generator = test_datagen.flow_from_directory(
    'Enter your path to test folder',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    shuffle=False)

# Feature extraction
def extract_features(generator, model_vgg16, model_inception):
    features_vgg16 = model_vgg16.predict(generator, verbose=1)
    features_inception = model_inception.predict(generator, verbose=1)
    features = np.concatenate([features_vgg16, features_inception], axis=1)
    labels = generator.classes
    return features, labels

train_features, train_labels = extract_features(train_generator, model_vgg16, model_inception)
test_features, test_labels = extract_features(test_generator, model_vgg16, model_inception)

# Encode labels
le = LabelEncoder()
train_labels_enc = le.fit_transform(train_labels)
test_labels_enc = le.transform(test_labels)

# Define classifiers
classifiers = {
    'Naive Bayes': GaussianNB(),
    'Linear SVM': SVC(kernel='linear', probability=True),
    'Quadratic SVM': SVC(kernel='poly', degree=2, probability=True),
    'Cubic SVM': SVC(kernel='poly', degree=3, probability=True),
    'Fine Gaussian SVM': SVC(kernel='rbf', gamma='scale', C=0.1, probability=True),
    'Medium Gaussian SVM': SVC(kernel='rbf', gamma='scale', C=1, probability=True),
    'Coarse Gaussian SVM': SVC(kernel='rbf', gamma='scale', C=10, probability=True),
    'Fine KNN': KNeighborsClassifier(n_neighbors=1),
    'Medium KNN': KNeighborsClassifier(n_neighbors=5),
    'Coarse KNN': KNeighborsClassifier(n_neighbors=10),
    'Cosine KNN': KNeighborsClassifier(n_neighbors=5, metric='cosine'),
    'Cubic KNN': KNeighborsClassifier(n_neighbors=5, metric='minkowski', p=3),
    'Weighted KNN': KNeighborsClassifier(n_neighbors=5, weights='distance')
}

# Initialize results storage
results = {name: {'accuracy': [], 'auc': [], 'recall': [], 'precision': [], 'f1': []} for name in classifiers.keys()}
all_run_results = []

# Perform classification and record metrics
for name, clf in classifiers.items():
    print(f"Training and evaluating {name}...")
    for run in tqdm(range(10)):
        clf.fit(train_features, train_labels_enc)
        pred_labels = clf.predict(test_features)
        pred_probs = clf.predict_proba(test_features)[:, 1] if hasattr(clf, "predict_proba") else pred_labels

        accuracy = accuracy_score(test_labels_enc, pred_labels)
        auc = roc_auc_score(test_labels_enc, pred_probs)
        recall = recall_score(test_labels_enc, pred_labels)
        precision = precision_score(test_labels_enc, pred_labels)
        f1 = f1_score(test_labels_enc, pred_labels)

        results[name]['accuracy'].append(accuracy)
        results[name]['auc'].append(auc)
        results[name]['recall'].append(recall)
        results[name]['precision'].append(precision)
        results[name]['f1'].append(f1)

        all_run_results.append({
            'classifier': name,
            'run': run + 1,
            'accuracy': accuracy,
            'auc': auc,
            'recall': recall,
            'precision': precision,
            'f1': f1
        })

# Calculate mean and standard deviation
summary = {}
for name, metrics in results.items():
    summary[name] = {metric: {'mean': np.mean(values), 'std': np.std(values)} for metric, values in metrics.items()}

# Convert summary to DataFrame
summary_df = pd.DataFrame({(clf, metric): vals for clf, metrics in summary.items() for metric, vals in metrics.items()})
print(summary_df)

# Convert all run results to DataFrame
all_run_results_df = pd.DataFrame(all_run_results)

# Save results to CSV
summary_df.to_csv('classification_summary_vgg16_inception_fusion.csv')
all_run_results_df.to_csv('all_run_results_vgg16_inception_fusion.csv')

In [ ]:
#FUSION MODEL 4: VGG19 & Resnet50

import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, roc_auc_score, recall_score, precision_score, f1_score
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg19 import VGG19, preprocess_input as preprocess_input_vgg19
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input as preprocess_input_resnet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

# Load VGG19 model pre-trained on ImageNet without top layers
base_model_vgg19 = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x_vgg19 = base_model_vgg19.output
x_vgg19 = GlobalAveragePooling2D()(x_vgg19)
model_vgg19 = Model(inputs=base_model_vgg19.input, outputs=x_vgg19)

# Load ResNet50 model pre-trained on ImageNet without top layers
base_model_resnet50 = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x_resnet50 = base_model_resnet50.output
x_resnet50 = GlobalAveragePooling2D()(x_resnet50)
model_resnet50 = Model(inputs=base_model_resnet50.input, outputs=x_resnet50)

# Prepare data generators
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input_vgg19)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input_vgg19)

train_generator = train_datagen.flow_from_directory(
    'Enter your path to train folder',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    shuffle=False)

test_generator = test_datagen.flow_from_directory(
    'Enter your path to test folder',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    shuffle=False)

# Feature extraction
def extract_features(generator, model_vgg19, model_resnet50):
    features_vgg19 = model_vgg19.predict(generator, verbose=1)
    features_resnet50 = model_resnet50.predict(generator, verbose=1)
    features = np.concatenate([features_vgg19, features_resnet50], axis=1)
    labels = generator.classes
    return features, labels

train_features, train_labels = extract_features(train_generator, model_vgg19, model_resnet50)
test_features, test_labels = extract_features(test_generator, model_vgg19, model_resnet50)

# Encode labels
le = LabelEncoder()
train_labels_enc = le.fit_transform(train_labels)
test_labels_enc = le.transform(test_labels)

# Define classifiers
classifiers = {
    'Naive Bayes': GaussianNB(),
    'Linear SVM': SVC(kernel='linear', probability=True),
    'Quadratic SVM': SVC(kernel='poly', degree=2, probability=True),
    'Cubic SVM': SVC(kernel='poly', degree=3, probability=True),
    'Fine Gaussian SVM': SVC(kernel='rbf', gamma='scale', C=0.1, probability=True),
    'Medium Gaussian SVM': SVC(kernel='rbf', gamma='scale', C=1, probability=True),
    'Coarse Gaussian SVM': SVC(kernel='rbf', gamma='scale', C=10, probability=True),
    'Fine KNN': KNeighborsClassifier(n_neighbors=1),
    'Medium KNN': KNeighborsClassifier(n_neighbors=5),
    'Coarse KNN': KNeighborsClassifier(n_neighbors=10),
    'Cosine KNN': KNeighborsClassifier(n_neighbors=5, metric='cosine'),
    'Cubic KNN': KNeighborsClassifier(n_neighbors=5, metric='minkowski', p=3),
    'Weighted KNN': KNeighborsClassifier(n_neighbors=5, weights='distance')
}

# Initialize results storage
results = {name: {'accuracy': [], 'auc': [], 'recall': [], 'precision': [], 'f1': []} for name in classifiers.keys()}
all_run_results = []

# Perform classification and record metrics
for name, clf in classifiers.items():
    print(f"Training and evaluating {name}...")
    for run in tqdm(range(10)):
        clf.fit(train_features, train_labels_enc)
        pred_labels = clf.predict(test_features)
        pred_probs = clf.predict_proba(test_features)[:, 1] if hasattr(clf, "predict_proba") else pred_labels

        accuracy = accuracy_score(test_labels_enc, pred_labels)
        auc = roc_auc_score(test_labels_enc, pred_probs)
        recall = recall_score(test_labels_enc, pred_labels)
        precision = precision_score(test_labels_enc, pred_labels)
        f1 = f1_score(test_labels_enc, pred_labels)

        results[name]['accuracy'].append(accuracy)
        results[name]['auc'].append(auc)
        results[name]['recall'].append(recall)
        results[name]['precision'].append(precision)
        results[name]['f1'].append(f1)

        all_run_results.append({
            'classifier': name,
            'run': run + 1,
            'accuracy': accuracy,
            'auc': auc,
            'recall': recall,
            'precision': precision,
            'f1': f1
        })

# Calculate mean and standard deviation
summary = {}
for name, metrics in results.items():
    summary[name] = {metric: {'mean': np.mean(values), 'std': np.std(values)} for metric, values in metrics.items()}

# Convert summary to DataFrame
summary_df = pd.DataFrame({(clf, metric): vals for clf, metrics in summary.items() for metric, vals in metrics.items()})
print(summary_df)

# Convert all run results to DataFrame
all_run_results_df = pd.DataFrame(all_run_results)

# Save results to CSV
summary_df.to_csv('classification_summary_vgg19_resnet50_fusion.csv')
all_run_results_df.to_csv('all_run_results_vgg19_resnet50_fusion.csv')

In [ ]:
#FUSION MODEL 5: VGG19 & GoogleNet (Inception v3)

import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, roc_auc_score, recall_score, precision_score, f1_score
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg19 import VGG19, preprocess_input as preprocess_input_vgg19
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input as preprocess_input_inception
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

# Load VGG19 model pre-trained on ImageNet without top layers
base_model_vgg19 = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x_vgg19 = base_model_vgg19.output
x_vgg19 = GlobalAveragePooling2D()(x_vgg19)
model_vgg19 = Model(inputs=base_model_vgg19.input, outputs=x_vgg19)

# Load InceptionV3 (GoogLeNet) model pre-trained on ImageNet without top layers
base_model_inception = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x_inception = base_model_inception.output
x_inception = GlobalAveragePooling2D()(x_inception)
model_inception = Model(inputs=base_model_inception.input, outputs=x_inception)

# Prepare data generators
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input_vgg19)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input_vgg19)

train_generator = train_datagen.flow_from_directory(
    'Enter your path to train folder',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    shuffle=False)

test_generator = test_datagen.flow_from_directory(
    'Enter your path to test folder',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    shuffle=False)

# Feature extraction
def extract_features(generator, model_vgg19, model_inception):
    features_vgg19 = model_vgg19.predict(generator, verbose=1)
    features_inception = model_inception.predict(generator, verbose=1)
    features = np.concatenate([features_vgg19, features_inception], axis=1)
    labels = generator.classes
    return features, labels

train_features, train_labels = extract_features(train_generator, model_vgg19, model_inception)
test_features, test_labels = extract_features(test_generator, model_vgg19, model_inception)

# Encode labels
le = LabelEncoder()
train_labels_enc = le.fit_transform(train_labels)
test_labels_enc = le.transform(test_labels)

# Define classifiers
classifiers = {
    'Naive Bayes': GaussianNB(),
    'Linear SVM': SVC(kernel='linear', probability=True),
    'Quadratic SVM': SVC(kernel='poly', degree=2, probability=True),
    'Cubic SVM': SVC(kernel='poly', degree=3, probability=True),
    'Fine Gaussian SVM': SVC(kernel='rbf', gamma='scale', C=0.1, probability=True),
    'Medium Gaussian SVM': SVC(kernel='rbf', gamma='scale', C=1, probability=True),
    'Coarse Gaussian SVM': SVC(kernel='rbf', gamma='scale', C=10, probability=True),
    'Fine KNN': KNeighborsClassifier(n_neighbors=1),
    'Medium KNN': KNeighborsClassifier(n_neighbors=5),
    'Coarse KNN': KNeighborsClassifier(n_neighbors=10),
    'Cosine KNN': KNeighborsClassifier(n_neighbors=5, metric='cosine'),
    'Cubic KNN': KNeighborsClassifier(n_neighbors=5, metric='minkowski', p=3),
    'Weighted KNN': KNeighborsClassifier(n_neighbors=5, weights='distance')
}

# Initialize results storage
results = {name: {'accuracy': [], 'auc': [], 'recall': [], 'precision': [], 'f1': []} for name in classifiers.keys()}
all_run_results = []

# Perform classification and record metrics
for name, clf in classifiers.items():
    print(f"Training and evaluating {name}...")
    for run in tqdm(range(10)):
        clf.fit(train_features, train_labels_enc)
        pred_labels = clf.predict(test_features)
        pred_probs = clf.predict_proba(test_features)[:, 1] if hasattr(clf, "predict_proba") else pred_labels

        accuracy = accuracy_score(test_labels_enc, pred_labels)
        auc = roc_auc_score(test_labels_enc, pred_probs)
        recall = recall_score(test_labels_enc, pred_labels)
        precision = precision_score(test_labels_enc, pred_labels)
        f1 = f1_score(test_labels_enc, pred_labels)

        results[name]['accuracy'].append(accuracy)
        results[name]['auc'].append(auc)
        results[name]['recall'].append(recall)
        results[name]['precision'].append(precision)
        results[name]['f1'].append(f1)

        all_run_results.append({
            'classifier': name,
            'run': run + 1,
            'accuracy': accuracy,
            'auc': auc,
            'recall': recall,
            'precision': precision,
            'f1': f1
        })

# Calculate mean and standard deviation
summary = {}
for name, metrics in results.items():
    summary[name] = {metric: {'mean': np.mean(values), 'std': np.std(values)} for metric, values in metrics.items()}

# Convert summary to DataFrame
summary_df = pd.DataFrame({(clf, metric): vals for clf, metrics in summary.items() for metric, vals in metrics.items()})
print(summary_df)

# Convert all run results to DataFrame
all_run_results_df = pd.DataFrame(all_run_results)

# Save results to CSV
summary_df.to_csv('classification_summary_vgg19_inception_fusion_10000.csv')
all_run_results_df.to_csv('all_run_results_vgg19_inception_fusion_10000.csv')

In [ ]:
#FUSION MODEL 6: Resnet50 & GoogleNet (Inception v3)

import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, roc_auc_score, recall_score, precision_score, f1_score
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input as preprocess_input_resnet50
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input as preprocess_input_inception
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

# Load ResNet50 model pre-trained on ImageNet without top layers
base_model_resnet50 = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x_resnet50 = base_model_resnet50.output
x_resnet50 = GlobalAveragePooling2D()(x_resnet50)
model_resnet50 = Model(inputs=base_model_resnet50.input, outputs=x_resnet50)

# Load InceptionV3 (GoogLeNet) model pre-trained on ImageNet without top layers
base_model_inception = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x_inception = base_model_inception.output
x_inception = GlobalAveragePooling2D()(x_inception)
model_inception = Model(inputs=base_model_inception.input, outputs=x_inception)

# Prepare data generators
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input_resnet50)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input_resnet50)

train_generator = train_datagen.flow_from_directory(
    'Enter your path to train folder',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    shuffle=False)

test_generator = test_datagen.flow_from_directory(
    'Enter your path to test folder',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    shuffle=False)

# Feature extraction
def extract_features(generator, model_resnet50, model_inception):
    features_resnet50 = model_resnet50.predict(generator, verbose=1)
    features_inception = model_inception.predict(generator, verbose=1)
    features = np.concatenate([features_resnet50, features_inception], axis=1)
    labels = generator.classes
    return features, labels

train_features, train_labels = extract_features(train_generator, model_resnet50, model_inception)
test_features, test_labels = extract_features(test_generator, model_resnet50, model_inception)

# Encode labels
le = LabelEncoder()
train_labels_enc = le.fit_transform(train_labels)
test_labels_enc = le.transform(test_labels)

# Define classifiers
classifiers = {
    'Naive Bayes': GaussianNB(),
    'Linear SVM': SVC(kernel='linear', probability=True),
    'Quadratic SVM': SVC(kernel='poly', degree=2, probability=True),
    'Cubic SVM': SVC(kernel='poly', degree=3, probability=True),
    'Fine Gaussian SVM': SVC(kernel='rbf', gamma='scale', C=0.1, probability=True),
    'Medium Gaussian SVM': SVC(kernel='rbf', gamma='scale', C=1, probability=True),
    'Coarse Gaussian SVM': SVC(kernel='rbf', gamma='scale', C=10, probability=True),
    'Fine KNN': KNeighborsClassifier(n_neighbors=1),
    'Medium KNN': KNeighborsClassifier(n_neighbors=5),
    'Coarse KNN': KNeighborsClassifier(n_neighbors=10),
    'Cosine KNN': KNeighborsClassifier(n_neighbors=5, metric='cosine'),
    'Cubic KNN': KNeighborsClassifier(n_neighbors=5, metric='minkowski', p=3),
    'Weighted KNN': KNeighborsClassifier(n_neighbors=5, weights='distance')
}

# Initialize results storage
results = {name: {'accuracy': [], 'auc': [], 'recall': [], 'precision': [], 'f1': []} for name in classifiers.keys()}
all_run_results = []

# Perform classification and record metrics
for name, clf in classifiers.items():
    print(f"Training and evaluating {name}...")
    for run in tqdm(range(10)):
        clf.fit(train_features, train_labels_enc)
        pred_labels = clf.predict(test_features)
        pred_probs = clf.predict_proba(test_features)[:, 1] if hasattr(clf, "predict_proba") else pred_labels

        accuracy = accuracy_score(test_labels_enc, pred_labels)
        auc = roc_auc_score(test_labels_enc, pred_probs)
        recall = recall_score(test_labels_enc, pred_labels)
        precision = precision_score(test_labels_enc, pred_labels)
        f1 = f1_score(test_labels_enc, pred_labels)

        results[name]['accuracy'].append(accuracy)
        results[name]['auc'].append(auc)
        results[name]['recall'].append(recall)
        results[name]['precision'].append(precision)
        results[name]['f1'].append(f1)

        all_run_results.append({
            'classifier': name,
            'run': run + 1,
            'accuracy': accuracy,
            'auc': auc,
            'recall': recall,
            'precision': precision,
            'f1': f1
        })

# Calculate mean and standard deviation
summary = {}
for name, metrics in results.items():
    summary[name] = {metric: {'mean': np.mean(values), 'std': np.std(values)} for metric, values in metrics.items()}

# Convert summary to DataFrame
summary_df = pd.DataFrame({(clf, metric): vals for clf, metrics in summary.items() for metric, vals in metrics.items()})
print(summary_df)

# Convert all run results to DataFrame
all_run_results_df = pd.DataFrame(all_run_results)

# Save results to CSV
summary_df.to_csv('classification_summary_resnet50_inception_fusion.csv')
all_run_results_df.to_csv('all_run_results_resnet50_inception_fusion.csv')

In [ ]:
#FINAL PROPOSED MODEL : Fusion of VGG16 & VGG19 alongside handcrafted feature LBP

import os
import cv2
import numpy as np
import pandas as pd
from skimage.feature import local_binary_pattern
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, roc_auc_score, recall_score, precision_score, f1_score
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input as preprocess_input_vgg16
from tensorflow.keras.applications.vgg19 import VGG19, preprocess_input as preprocess_input_vgg19
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

# Load VGG16 model pre-trained on ImageNet without top layers
base_model_vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x_vgg16 = base_model_vgg16.output
x_vgg16 = GlobalAveragePooling2D()(x_vgg16)
model_vgg16 = Model(inputs=base_model_vgg16.input, outputs=x_vgg16)

# Load VGG19 model pre-trained on ImageNet without top layers
base_model_vgg19 = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x_vgg19 = base_model_vgg19.output
x_vgg19 = GlobalAveragePooling2D()(x_vgg19)
model_vgg19 = Model(inputs=base_model_vgg19.input, outputs=x_vgg19)

# Prepare data generators
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input_vgg16)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input_vgg16)

train_generator = train_datagen.flow_from_directory(
    'C://Users//sunri//Downloads//Modified_Dataset//train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    shuffle=False)

test_generator = test_datagen.flow_from_directory(
    'C://Users//sunri//Downloads//Modified_Dataset//test',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    shuffle=False)

# Extract LBP features
def extract_lbp_features(image_paths):
    lbp_features = []
    for img_path in image_paths:
        image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        image = cv2.resize(image, (224, 224))
        lbp = local_binary_pattern(image, P=8, R=1, method='uniform')
        hist, _ = np.histogram(lbp, bins=np.arange(0, 10), density=True)
        lbp_features.append(hist)
    return np.array(lbp_features)

# Feature extraction
def extract_features(generator, model_vgg16, model_vgg19):
    features_vgg16 = model_vgg16.predict(generator, verbose=1)
    features_vgg19 = model_vgg19.predict(generator, verbose=1)
    image_paths = [generator.filepaths[i] for i in range(generator.samples)]
    features_lbp = extract_lbp_features(image_paths)
    features = np.concatenate([features_vgg16, features_vgg19, features_lbp], axis=1)
    labels = generator.classes
    return features, labels

train_features, train_labels = extract_features(train_generator, model_vgg16, model_vgg19)
test_features, test_labels = extract_features(test_generator, model_vgg16, model_vgg19)

# Encode labels
le = LabelEncoder()
train_labels_enc = le.fit_transform(train_labels)
test_labels_enc = le.transform(test_labels)

# Define classifiers
classifiers = {
    'Naive Bayes': GaussianNB(),
    'Linear SVM': SVC(kernel='linear', probability=True),
    'Quadratic SVM': SVC(kernel='poly', degree=2, probability=True),
    'Cubic SVM': SVC(kernel='poly', degree=3, probability=True),
    'Fine Gaussian SVM': SVC(kernel='rbf', gamma='scale', C=0.1, probability=True),
    'Medium Gaussian SVM': SVC(kernel='rbf', gamma='scale', C=1, probability=True),
    'Coarse Gaussian SVM': SVC(kernel='rbf', gamma='scale', C=10, probability=True),
    'Fine KNN': KNeighborsClassifier(n_neighbors=1),
    'Medium KNN': KNeighborsClassifier(n_neighbors=5),
    'Coarse KNN': KNeighborsClassifier(n_neighbors=10),
    'Cosine KNN': KNeighborsClassifier(n_neighbors=5, metric='cosine'),
    'Cubic KNN': KNeighborsClassifier(n_neighbors=5, metric='minkowski', p=3),
    'Weighted KNN': KNeighborsClassifier(n_neighbors=5, weights='distance')
}

# Initialize results storage
results = {name: {'accuracy': [], 'auc': [], 'recall': [], 'precision': [], 'f1': []} for name in classifiers.keys()}
all_run_results = []

# Perform classification and record metrics
for name, clf in classifiers.items():
    print(f"Training and evaluating {name}...")
    for run in tqdm(range(10)):
        clf.fit(train_features, train_labels_enc)
        pred_labels = clf.predict(test_features)
        pred_probs = clf.predict_proba(test_features)[:, 1] if hasattr(clf, "predict_proba") else pred_labels

        accuracy = accuracy_score(test_labels_enc, pred_labels)
        auc = roc_auc_score(test_labels_enc, pred_probs)
        recall = recall_score(test_labels_enc, pred_labels)
        precision = precision_score(test_labels_enc, pred_labels)
        f1 = f1_score(test_labels_enc, pred_labels)

        results[name]['accuracy'].append(accuracy)
        results[name]['auc'].append(auc)
        results[name]['recall'].append(recall)
        results[name]['precision'].append(precision)
        results[name]['f1'].append(f1)

        all_run_results.append({
            'classifier': name,
            'run': run + 1,
            'accuracy': accuracy,
            'auc': auc,
            'recall': recall,
            'precision': precision,
            'f1': f1
        })

# Calculate mean and standard deviation
summary = {}
for name, metrics in results.items():
    summary[name] = {metric: {'mean': np.mean(values), 'std': np.std(values)} for metric, values in metrics.items()}

# Convert summary to DataFrame
summary_df = pd.DataFrame({(clf, metric): vals for clf, metrics in summary.items() for metric, vals in metrics.items()})
print(summary_df)

# Convert all run results to DataFrame
all_run_results_df = pd.DataFrame(all_run_results)

# Save results to CSV
summary_df.to_csv('classification_summary_vgg16_vgg19_lbp_fusion.csv')
all_run_results_df.to_csv('all_run_results_vgg16_vgg19_lbp_fusion.csv')